# Data Extraction
This notebook uses the Domain developer API to extract data and create various datasets

In [1]:
import pandas as pd
import requests
import json
PROJECT_PATH = '/Users/elmi/Projects/HouseHunt'

headers = {
    'X-Api-Key': 'key_aa83d5090fa4db8cd2ddbad271af541f'
}

state = 'QLD'
property_category = 'unit'
num_bedrooms = 2
period_size = 'years' #halfYears, quarters
num_periods = 5


In [3]:
suburb='Moorooka'
r = requests.get(f"https://api.domain.com.au/v2/suburbPerformanceStatistics/{state}/{suburb}/%7Bpostcode%7D?propertyCategory={property_category}&bedrooms={num_bedrooms}&periodSize={period_size}&startingPeriodRelativeToCurrent=1&totalPeriods={num_periods}", headers=headers)
r.text

'{\n  "type": "https://tools.ietf.org/html/rfc7231#section-6.6.1",\n  "title": "An error occurred while processing your request.",\n  "status": 500,\n  "traceId": "00-71c6bb768438b34ba7144add639b7c54-9fd5d1b3622dd749-01"\n}'

In [8]:
f"https://api.domain.com.au/v2/suburbPerformanceStatistics/{state}/{suburb}/%7Bpostcode%7D?propertyCategory={property_category}&bedrooms={num_bedrooms}&periodSize={period_size}&startingPeriodRelativeToCurrent=1&totalPeriods={num_periods}"

'https://api.domain.com.au/v2/suburbPerformanceStatistics/QLD/Moorooka/%7Bpostcode%7D?propertyCategory=unit&bedrooms=2&periodSize=years&startingPeriodRelativeToCurrent=1&totalPeriods=5'

## Suburb Performance Stats

In [102]:
# Read raw addressLocators API data
suburbs = pd.read_csv(f"{PROJECT_PATH}/data/raw/20180105__brisbane_190_suburbs_list_.csv", header=None)[0].to_list()

api_response = []

for suburb in suburbs:
    r = requests.get(f"https://api.domain.com.au/v2/suburbPerformanceStatistics/{state}/{suburb}/%7Bpostcode%7D?propertyCategory={property_category}&bedrooms={num_bedrooms}&periodSize={period_size}&startingPeriodRelativeToCurrent=1&totalPeriods={num_periods}", headers=headers)
    if not r.ok:
        print(f'Skipping {suburb}\n {r.text}')
    api_response.extend(json.loads(r.text))

with open(f'{PROJECT_PATH}/data/raw/api_suburbPerformanceStatistics.json', 'w') as f:
    json.dump(api_response, f)

api_response[0].keys()

Skipping Acacia Ridge
 {
  "type": "https://tools.ietf.org/html/rfc7231#section-6.6.1",
  "title": "An error occurred while processing your request.",
  "status": 500,
  "traceId": "00-defe3a93d4f6b342913f7a6e30b9aee5-b75253d0295b0448-01"
}
Skipping Albion
 {
  "type": "https://tools.ietf.org/html/rfc7231#section-6.6.1",
  "title": "An error occurred while processing your request.",
  "status": 500,
  "traceId": "00-c3524dca0c5e15469b78c088f967ddf9-794594cddd4fc642-01"
}
Skipping Alderley
 {
  "type": "https://tools.ietf.org/html/rfc7231#section-6.6.1",
  "title": "An error occurred while processing your request.",
  "status": 500,
  "traceId": "00-94df6e696ae8ca409e2b2d0b6ca81131-d8c105f120271a44-01"
}
Skipping Algester
 {
  "type": "https://tools.ietf.org/html/rfc7231#section-6.6.1",
  "title": "An error occurred while processing your request.",
  "status": 500,
  "traceId": "00-d64a6c69e428c244852a660dba7f9672-01487524708dac44-01"
}
Skipping Annerley
 {
  "type": "https://tools.ietf

In [99]:
# Process suburb_id, property_category, year, month, stats
api_response = json.load(open(f'{PROJECT_PATH}/data/raw/api_suburbPerformanceStatistics.json'))
api_processed = []
for r in api_response:
    suburb = r['header']['suburb']
    state = r['header']['state']
    property_category = r['header']['propertyCategory']
    out = []
    for s in r['series']['seriesInfo']:
        row = {'suburb': suburb, 'state': state, 'property_category': property_category, 'year': s['year'], 'month': s['month']} | s['values']
        out.append(row)
    api_processed.extend(out)

pd.DataFrame(api_processed).to_csv(f"{PROJECT_PATH}/data/processed/suburb_performance_stats.csv", index=False)

api_processed[0].keys()  

dict_keys(['suburb_id', 'suburb', 'property_category', 'year', 'month', 'medianSoldPrice', 'numberSold', 'highestSoldPrice', 'lowestSoldPrice', '5thPercentileSoldPrice', '25thPercentileSoldPrice', '75thPercentileSoldPrice', '95thPercentileSoldPrice', 'medianSaleListingPrice', 'numberSaleListing', 'highestSaleListingPrice', 'lowestSaleListingPrice', 'auctionNumberAuctioned', 'auctionNumberSold', 'auctionNumberWithdrawn', 'daysOnMarket', 'discountPercentage', 'medianRentListingPrice', 'numberRentListing', 'highestRentListingPrice', 'lowestRentListingPrice'])

curl -X GET "https://api.domain.com.au/v1/suburbPerformanceStatistics?state=QLD&suburbId=104&propertyCategory=unit&chronologicalSpan=3&tPlusFrom=1&tPlusTo=20" -H "accept: application/json" -H "X-Api-Key: key_aa83d5090fa4db8cd2ddbad271af541f"